In [1]:
from utils import *
from prepare_ds import *
from train_ml import *
from create_map import *

Initialization paths...
All paths was initialized.


In [1]:
percent = 0.05
z_x, z_y, x, y = iteration_dataset(percent=percent)


NameError: name 'iteration_dataset' is not defined

In [3]:
m = train_model(z_x, z_y)
m

Split X, y -> X_train, y_train...
Start training model...
Selected model / method: RF
Model was trained. Start validate it...

Classification Report:
              precision    recall  f1-score   support

           1       0.65      0.67      0.66      1481
           2       0.66      0.78      0.72      5212
           3       0.47      0.38      0.42      3336
           4       0.81      0.63      0.71       421
           5       0.50      0.15      0.23        74
           6       0.13      0.01      0.02       247

    accuracy                           0.61     10771
   macro avg       0.54      0.44      0.46     10771
weighted avg       0.59      0.61      0.60     10771

Confusion Matrix:
[[ 995  179  290   11    1    5]
 [ 132 4071  995    6    5    3]
 [ 336 1694 1279   21    4    2]
 [  42   28   82  265    1    3]
 [   8   17   35    3   11    0]
 [  21  137   66   21    0    2]]
Saving model...
Model was saved: /Users/stephenhawking/Coding/ML/low2high_map/src/../data/

RandomForestClassifier(random_state=42)

In [4]:
# Mapping by RAM tensor-images (230m)
n = f"handle_map_{percent}.tif"
create_map(x, m, n)

Reshaping tensor-images...
Start create of map...

map size: x = 1421, y = 1349, bands = 4, total px=1916929


Creating map...: 100%|██████████| 2/2 [00:14<00:00,  7.16s/it]

Map is done.
Data saved to /Users/stephenhawking/Coding/ML/low2high_map/src/../data/output/handle_map_0.05.tif


In [5]:
# Mapping by loaded images from path (10m)
p = DEFAULT_PATH["images"]
n = f"handle_map_10m_{percent}.tif"
images = load_data_tif(p)

Loading file: /Users/stephenhawking/Coding/ML/low2high_map/src/../data/input/images_10m/Sentinel_Samara/2023_05_0510_comp.b2.tif
Loading file: /Users/stephenhawking/Coding/ML/low2high_map/src/../data/input/images_10m/Sentinel_Samara/2023_05_0510_comp.b3.tif
Loading file: /Users/stephenhawking/Coding/ML/low2high_map/src/../data/input/images_10m/Sentinel_Samara/2023_05_0510_comp.b4.tif
Loading file: /Users/stephenhawking/Coding/ML/low2high_map/src/../data/input/images_10m/Sentinel_Samara/2023_05_0510_comp.b8.tif
Files was loaded.


In [7]:
create_map(images, m, n)

Reshaping tensor-images...
Start create of map...

map size: x = 32664, y = 31014, bands = 4, total px=1013041296


Creating map...: 100%|██████████| 967/967 [1:50:39<00:00,  6.87s/it]  


Map is done.


ERROR 1: TIFFSetField:/Users/stephenhawking/Coding/ML/low2high_map/src/../data/output/handle_map_10m_0.05.tif: Cannot modify tag "PhotometricInterpretation" while writing


Data saved to /Users/stephenhawking/Coding/ML/low2high_map/src/../data/output/handle_map_10m_0.05.tif
